In [1]:
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import gensim, logging
import numpy as np
import random

/Users/francesco/miniconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
def make_score_files(model, filelabel):
    with open(filelabel + '_scores.csv', 'w') as scorefile:
        with open(filelabel + '_words.csv', 'w') as wordfile:
            count_scores = 0
            count_words = 0
            for i in range(len(model.index2word)):
                word = model.index2word[i]
                if i == 80:
                    continue
                try:
                    wordfile.write(word + '\n')
                    score = model[word]
                    scores = [str(x) for x in score]
                    scorefile.write('\t'.join(scores) + '\n')
                    #print("word: " + word)
                    count_scores += 1
                    count_words += 1
                except:
                    print "Not found: " + str(i) + ": " + word
                    continue
            print("counts: scores = " + str(count_scores) + "; words = " + str(count_words))     
            #scorefile.write('\n')
            #wordfile.write('\n')    

In [3]:
model = gensim.models.Word2Vec.load('pycode/bigram.txt.doc2vec.word_model')

In [4]:
stop_words = [line.strip('\n') for line in open("pycode/stop_words.txt", 'r')]jjj

In [5]:
top_characters = ['luke_skywalker','han_solo','princess_leia','obiwan','chewbacca','vader','anakin_skywalker','yoda','boba_fett','emperor']

In [6]:
main_neighbors = []
for c in top_characters:
    main_neighbors.append(c)
    l = model.most_similar(positive=c, topn=10)
    for (n,v) in l:
        main_neighbors.append(n)

In [7]:
def make_score_files_cons(model, stop_words, filelabel):
    with open(filelabel + '_scores.csv', 'w') as scorefile:
        with open(filelabel + '_words.csv', 'w') as wordfile:
            count_scores = 0
            count_words = 0
            for i in range(len(model.index2word)):
                word = model.index2word[i]
                if word in stop_words:
                    #print("found stop word: " + word)
                    continue
                try:
                    wordfile.write(word + '\n')
                    score = model[word]
                    scores = [str(x) for x in score]
                    scorefile.write('\t'.join(scores) + '\n')
                    #print("word: " + word)
                    count_scores += 1
                    count_words += 1
                except:
                    print "Not found: " + str(i) + ": " + word
                    continue
            print("counts: scores = " + str(count_scores) + "; words = " + str(count_words))     
            #scorefile.write('\n')
            #wordfile.write('\n')   

In [8]:
make_score_files_cons(model, stop_words, 'mixed_sw_random')

counts: scores = 18599; words = 18599


In [9]:
# visualization
X = np.loadtxt("mixed_sw_random_scores.csv")
labels = np.genfromtxt("mixed_sw_random_words.csv", dtype=str)

In [10]:
weighted_idx = [ model.vocab[l].count for l in labels ]

In [11]:
def weighted_choice_sub(weights):
    rnd = random.random() * sum(weights)
    for i, w in enumerate(weights):
        rnd -= w
        if rnd < 0:
            return i

In [12]:
def add_characters(model, characters):
    indices = []
    for c in characters: 
        idx = model.index2word.index(c)
        if idx not in indices:
            indices.append(idx)
    return indices

In [13]:
def get_indices_cons(model, weights, characters, X, N):
    #indices = []
    indices = add_characters(model, characters)
    size = len(indices)
    while size != N:
        idx = weighted_choice_sub(weights)
        if idx not in indices:
            too_small = False
            for x in X[idx]: 
                if abs(x) < 0.0001: 
                    too_small = True
                    break
            if too_small: 
                continue
            indices.append(idx)
            size += 1
    return indices

In [14]:
random.seed(587458)
indices = get_indices_cons(model, weighted_idx, main_neighbors, X, 3000)

In [15]:
## check for doubles
count = {}
for i in indices:
    if i not in count.keys():
        count[i] = 1
    else:
        count[i] +=1
        if count[i] > 1:
            print(str(i) + " is duplicated " + str(count[i]) + " times")

In [16]:
l = []
scale = 1.0
for x in X[indices]:
    x_m = [xi for xi in x]
    l.append(x_m)
X_sub = np.array(l)

In [17]:
for i, x in enumerate(X_sub):
    for xi in x: 
        if abs(xi) <= 0.0001: 
            if i in indices:
                print(str(i) + " is in indices")

In [18]:
labels_sub = labels[indices]

In [19]:
np.savetxt('labels.csv', labels_sub,fmt='%s',delimiter=',')

In [20]:
np.savetxt('values.csv',X_sub, delimiter=',')

In [19]:
import tsne
Y = tsne.tsne(X_sub, 2, 50, 20.0)

Preprocessing the data using PCA...
Computing pairwise distances...
Computing P-values for point  0  of  3000 ...
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generates nan values
row 214 generate

KeyboardInterrupt: 